### 1. SQLite

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("datafile.db")
cursor = conn.cursor()

In [3]:
cursor.execute("create table people (id integer primary key, name text, count integer)")

In [4]:
cursor.execute("insert into people (name, count) values ('Bob', 1)")
cursor.execute("insert into people (name, count) values (?, ?)", ("Jill", 15))

In [5]:
conn.commit()

In [6]:
cursor.execute("insert into people (name, count) values (:username, :usercount)", {"username": "Joe", "usercount": 10})

In [7]:
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 15), (3, 'Joe', 10)]


In [8]:
result = cursor.execute("select * from people where name like :name", {"name": "bob"})
print(result.fetchall())

[(1, 'Bob', 1)]


In [9]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]


In [10]:
result = cursor.execute("select * from people")
for row in result:
    print(row)

(1, 'Bob', 1)
(2, 'Jill', 20)
(3, 'Joe', 10)


In [11]:
conn.commit()
conn.close()

### 2. Postgresql

In [12]:
import psycopg2
from psycopg2 import OperationalError

In [13]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name, 
            user=db_user, 
            password=db_password, 
            host=db_host, 
            port=db_port)
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [14]:
def create_database(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [15]:
connection = create_connection("postgres", "postgres", "****", "127.0.0.1", "5432")

Connection to PostgreSQL DB successful


In [17]:
create_database_query = "CREATE DATABASE dbpython"
create_database(connection, create_database_query)

Query executed successfully


In [18]:
connection = create_connection("dbpython", "postgres", "****", "127.0.0.1", "5432")

Connection to PostgreSQL DB successful


In [19]:
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [20]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
 id SERIAL PRIMARY KEY,
 name TEXT NOT NULL,
 age INTEGER,
 gender TEXT,
 nationality TEXT
)
"""
execute_query(connection, create_users_table)

Query executed successfully


In [21]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts (
 id SERIAL PRIMARY KEY,
 title TEXT NOT NULL,
 description TEXT NOT NULL,
 user_id INTEGER REFERENCES users(id)
)
"""
execute_query(connection, create_posts_table)

Query executed successfully


In [22]:
users = [
    ("James", 25, "male", "USA"),
    ("Leila", 32, "female", "France"),
    ("Brigitte", 35, "female", "England"),
    ("Mike", 40, "male", "Denmark"),
    ("Elizabeth", 21, "female", "Canada")]
user_records = ", ".join(["%s"] * len(users))
insert_query = (f"INSERT INTO users (name, age, gender, nationality) VALUES {user_records}")
connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, users)

In [23]:
posts = [
    ("Happy", "I am feeling very happy today", 1),
    ("Hot Weather", "The weather is very hot today", 2),
    ("Help", "I need some help with my work", 2),
    ("Great News", "I am getting married", 1),
    ("Interesting Game", "It was a fantastic game of tennis", 5),
    ("Party", "Anyone up for a late-night party today?", 3)]
post_records = ", ".join(["%s"] * len(posts))
insert_query = (f"INSERT INTO posts (title, description, user_id) VALUES {post_records}")
connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, posts)

In [24]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [25]:
select_users = "SELECT * FROM users"
users = execute_read_query(connection, select_users)
for user in users:
    print(user)

(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


### 3. MongoDB

In [26]:
import pymongo
import datetime
from bson.objectid import ObjectId

In [27]:
mongo = pymongo.MongoClient(host='localhost', port=27017)

In [28]:
db = mongo.my_data
collection = db.docs 
collection.find_one() 
db.list_collection_names()

[]

In [29]:
a_document = {'name': 'Jane',
              'age': 34,
              'interests': ['Python', 'databases', 'statistics'],
              'date_added': datetime.datetime.now()}

In [30]:
collection.insert_one(a_document)

In [31]:
db.list_collection_names()

['docs']

In [32]:
collection.find_one()

{'_id': ObjectId('5fa97602fb97c7180264efa4'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2020, 11, 9, 20, 1, 53, 954000)}

In [35]:
collection.find_one({"_id": ObjectId('5fa97602fb97c7180264efa4')}) 

{'_id': ObjectId('5fa97602fb97c7180264efa4'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2020, 11, 9, 20, 1, 53, 954000)}

In [36]:
collection.update_one({"_id": ObjectId('5fa97602fb97c7180264efa4')}, {"$set": {"name": "Ann"}})

In [37]:
collection.replace_one({"_id": ObjectId('5fa97602fb97c7180264efa4')}, {"name": "Ann"})

In [38]:
collection.find_one({"_id": ObjectId('5fa97602fb97c7180264efa4')})

{'_id': ObjectId('5fa97602fb97c7180264efa4'), 'name': 'Ann'}

In [39]:
collection.delete_one({"_id": ObjectId('5fa97602fb97c7180264efa4')})

In [40]:
collection.find_one()

In [41]:
db.list_collection_names()

['docs']

In [42]:
collection.drop()
db.list_collection_names()

[]